In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from matplotlib import dates
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from IPython.display import HTML
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Read in the radar data
radar_name = 'KGWX'
radar_type= 'NEXRAD'
elev_angle = 0.5
elev_str = str(elev_angle).replace('.', 'p')
elev_tol = 0.1

# For 03/30/22 case (PERiLS-2022 IOP2)
date = '0330'
radar_start_datetimestamp = '20220330220424'
radar_end_datetimestamp = '20220331035633'

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

radar_basedir = \
    '/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/IOP2/KGWX'
#radar_basedir = os.path.join(radar_basedir, '{}/{}'.format(date, radar_name[1:]))
radar_input_dir = radar_basedir
radar_output_dir = os.path.join(radar_basedir, 'extracted_sweeps')
if not os.path.exists(radar_output_dir):
    os.makedirs(radar_output_dir)

radar_start_timestamp = datetime_start.strftime('%H%M%S')
radar_end_timestamp = datetime_end.strftime('%H%M%S')
radar_combined_filename = f'{radar_name}_{radar_start_timestamp}_{radar_end_timestamp}_{elev_str}deg.nc'

radar_combined_filepath = os.path.join(radar_output_dir, radar_combined_filename)

# Read in individual radar volumes, extract the low-level sweepsstack them into a combined Dataset and dump to 
# disk
radar_paths = glob.glob(radar_input_dir + f'/{radar_name}*V06.nc')

radar_paths = sorted(radar_paths)
radar_input_list = []

for path in radar_paths:
    filename = os.path.basename(path)
    file_timestamp = filename[4:19]
    # print(file_timestamp)
    file_datetime = datetime.strptime(file_timestamp, '%Y%m%d_%H%M%S')
    if file_datetime >= datetime_start and file_datetime <= datetime_end:
        radar_input_list.append(path)

for radar_path in radar_input_list:
    print(f"Reading {os.path.basename(radar_path)}")
    radarobj = pyart.io.read(radar_path)
    print(f"Extracting any {elev_angle:.1f} deg sweeps...")
    elev_angles = radarobj.fixed_angle['data']
    elev_diffs = np.abs(elev_angles - elev_angle)
    req_sweeps = np.where(elev_diffs < elev_tol)[0]
    print(f"There are {len(req_sweeps)} sweeps within {elev_tol:.1f} deg of the target angle")
    print("Extracting to new radar objects and saving to files")
    for i, sweep_index in enumerate(req_sweeps):
        radarsweep = radarobj.extract_sweeps([sweep_index])
        sweep_time = pyart.graph.common.generate_radar_time_sweep(radarobj, i)
        time_string = "Time of sweep #{:d}: {}".format(i, sweep_time.strftime('%Y-%m-%d %H:%M:%S'))
        print(time_string)
        filetime_string = sweep_time.strftime("%Y%m%d_%H%M%S")
        filename = f"{radar_name}{filetime_string}_el{elev_str}.nc"
        filepath = os.path.join(radar_output_dir, filename)
        print(f"Saving {filename}...")
        pyart.io.write_cfradial(filepath, radarsweep)